In [16]:
''' 

[키워드 입력시 국내주식 중 관계가 가장 높은 종목 찾기]
    # 특정 키워드의 Google Trend 추이와 종목별 주가 추이와의 상관관계 분석
  
     >> 유의사항 : 아래 본인의 구글계정을 입력하고 실행하세요.
     
'''

import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import random
import time

# 키워드 입력 : 사드, 금리, 유가, 코스피, 미세먼지, 문재인 등
keyword='사드'

# 검색 기간입력 (2017년 1월 1일~)
time_term = '2017-01-01 2017-05-21'
start = datetime(2017, 1, 1)
end   = datetime(2017, 5, 21)

# 구글계정 입력 
google_username = "********@gmail.com"
google_password = "********"

In [17]:
# 구글트렌드 데이터 가져오기
from pytrends.request import TrendReq
pytrend = TrendReq(google_username, google_password, custom_useragent='Python Trends Script')
pytrend.build_payload(kw_list=[keyword], timeframe=time_term)
interest_over_time_df = pytrend.interest_over_time()
interest_over_time_df.head()

,사드
date,
2017-01-01,4
2017-01-02,6
2017-01-03,4
2017-01-04,6
2017-01-05,11


In [18]:
# 구글 국내주식 일자별가격 가져오기 (Sample : 종목 20개 -> 종목수 증대 Ver Update 필요)
kor_air = web.DataReader('KRX:003490','google',start,end)['Close']
kt = web.DataReader('KRX:030200','google',start,end)['Close']
lgchem = web.DataReader('KRX:051910','google',start,end)['Close']
samsung_el = web.DataReader('KRX:005930','google',start,end)['Close']
amoreg = web.DataReader('KRX:002790','google',start,end)['Close']
kbbank = web.DataReader('KRX:105560','google',start,end)['Close']
naver = web.DataReader('KRX:035420','google',start,end)['Close']
miraedw = web.DataReader('KRX:006800','google',start,end)['Close']
lotteshop = web.DataReader('KRX:023530','google',start,end)['Close']
soil = web.DataReader('KRX:010950','google',start,end)['Close']
ottogi = web.DataReader('KRX:007310','google',start,end)['Close']
kor_gas = web.DataReader('KRX:036460','google',start,end)['Close']
kia_car = web.DataReader('KRX:000270','google',start,end)['Close']
orion = web.DataReader('KRX:001800','google',start,end)['Close']
hana_finan = web.DataReader('KRX:086790','google',start,end)['Close']
posco = web.DataReader('KRX:005490','google',start,end)['Close']
emart = web.DataReader('KRX:139480','google',start,end)['Close']
wooribank = web.DataReader('KRX:000030','google',start,end)['Close']
samsung_secu = web.DataReader('KRX:016360','google',start,end)['Close']
isdongseo = web.DataReader('KRX:010780','google',start,end)['Close']

# 데이터 병합
data = pd.concat([interest_over_time_df,
                  kor_air,kt,lgchem,samsung_el,amoreg,
                  kbbank,naver,miraedw,lotteshop,soil,
                  ottogi,kor_gas,kia_car,orion,hana_finan,
                  posco,emart,wooribank,samsung_secu,isdongseo], axis=1)

# 컬럼 지정
data.columns = ['google_keyword',
                'kor_air','kt','lgchem','samsung_el','amoreg',
                'kbbank','naver','miraedw','lotteshop','soil',
                'ottogi','kor_gas','kia_car','orion','hana_finan',
                'posco','emart','wooribank','samsung_secu','isdongseo']

data.head()

,google_keyword,kor_air,kt,lgchem,samsung_el,amoreg,kbbank,naver,miraedw,lotteshop,...,ottogi,kor_gas,kia_car,orion,hana_finan,posco,emart,wooribank,samsung_secu,isdongseo
2017-01-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02,6,26750.0,29650.0,253000.0,1805000.0,122500.0,42600.0,777000.0,7400.0,216500.0,...,653000.0,48300.0,39500.0,624000.0,30900.0,260500.0,180500.0,12600.0,31450.0,43800.0
2017-01-03,4,27100.0,29300.0,262000.0,1824000.0,126000.0,43700.0,767000.0,7670.0,215000.0,...,649000.0,47600.0,40750.0,622000.0,31450.0,256000.0,179500.0,12700.0,31800.0,42950.0
2017-01-04,6,27050.0,29300.0,268500.0,1808000.0,127000.0,44150.0,810000.0,7980.0,209500.0,...,643000.0,47850.0,41300.0,621000.0,32000.0,254500.0,179000.0,12700.0,32450.0,42100.0
2017-01-05,11,27400.0,29200.0,267500.0,1778000.0,126000.0,42800.0,791000.0,8030.0,213500.0,...,644000.0,47200.0,41100.0,633000.0,31600.0,247500.0,186500.0,12300.0,32650.0,42900.0


In [19]:
# 로그수익률 계산
rets = np.log(data / data.shift(1))

# 상과계수 계산
df_corr = rets.corr()

# 결과물
def corr_bottom(code, bottom_cnt=-10):
     return df_corr[df_corr.index == code].T\
                 .sort_values(by=code, ascending=False)\
                 .ix[-bottom_cnt:]                  
corr_bottom('google_keyword', 30)

,google_keyword
google_keyword,1.000000
wooribank,0.141564
emart,0.136226
samsung_secu,0.135687
kia_car,0.111139
kt,0.080199
naver,0.076257
miraedw,0.069640
hana_finan,0.060770
ottogi,0.035002


In [ ]:
''' 시사점


   1. 구글트렌트 입력키워드 "사드"
   
      > 상관계수가 음의 방향(사드의 언급과 완전히 반대 방향)으로 움직인 종목은 '오리온' '롯데쇼핑' 등 중국 매출 기업
        : 시장 참여자들이 '사드' 로 인해 어떤 기업이 피해를 입을 것이라고 판단하는지 예측 가능함
        
      > 상관계수가 양의 방향으로 움직인 종목은 사드와 관계없는 내수기업 'KT' '이마트' 등
      
      > BUT! 키워드 상관관계 결과값으로 모든 주가의 움직임을 설명할 수 있는 것은 아님 

 
   2. 비즈니스 아이디어 제안
   
      > 투자자가 특정 주제 또는 키워드로 투자 정보를 찾고자 할 때 유용한 Tool 이 될 수 있음
      
      > 국내에 "미세먼지 증가" 로 인한 상장기업 중 관계가 있는 종목을 한 눈에 확인할 수 있음
      
      > 반도체 호황이 오면 일반적으로 투자자들은 '삼성전자' 'SK하이닉스' 를 떠올리지만, 반도체의 트렌드와 함께 움직이는
        기타 반도체 관련 기업들(부품, 장비, 유통 등)을 찾아낼 수도 있음
        
        
   3. 프로그램 Version Update 필요
   
      > 종목 Pool 확대 필요 (거래소, 코스닥 등) : 만들어보려 노력했으나 DATA 불러올 때마다 실패해서 포기하였음ㅠㅠ
   
      > 시점 조정 필요 : 구글트렌드는 주말에도 값이 나오지만, 일별 주가는 휴장일 데이터가 없음.. 이 부분 조정하여 정확도를 올려야 함
      
      > 기타 차트 구성 및 편의 기능 추가      
      
'''